<a href="https://colab.research.google.com/github/Beard31/Deep-Dive-Data-Science-Projects/blob/main/CapStoneBeard_4_31_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import geopandas as gpd
import json
import numpy as np
import pandas as pd
import requests
import plotly.express as px
import geopandas as gpd
import geopy.distance
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster

from shapely.geometry import Point


In [2]:
raw_text = """
Table: Schools in the Southeast Quadrant
Albuquerque High School
Albuquerque High
Jefferson Middle
Washington Middle
Dolores Gonzales Elementary
East San Jose Elementary
Eugene Field Elementary
Lew Wallace Elementary
Longfellow Elementary
Lowell Elementary
Monte Vista Elementary
Montezuma Elementary
Reginald Chavez Elementary
Zia Elementary
Highland High School
Highland High
Van Buren Middle
Wilson Middle
Bandelier Elementary
Emerson Elementary
Hawthorne Elementary
Kirtland Elementary
La Mesa Elementary
Manzano Mesa Elementary
Mark Twain Elementary
Sandia Base Elementary
Wherry Elementary
Whittier Elementary
Manzano High School
Manzano High
Jackson Middle
Kennedy Middle
Roosevelt Middle
A. Montoya Elementary
Apache Elementary
Chelwood Elementary
Collet Park Elementary
McCollum Elementary
Tomasita Elementary
Schools of Choice
Vision Quest
Table: Schools in the Southwest Quadrant
Atrisco Heritage High School
Atrisco Heritage Academy High
Truman Middle
Rudolfo Anaya Elementary
Helen Cordero Elementary
Edward Gonzales Elementary
Carlos Rey Elementary
George I. Sánchez (K-8)
Rio Grande High School
Rio Grande High
Ernie Pyle Middle
Harrison Middle
Polk Middle
Adobe Acres Elementary
Armijo Elementary
Atrisco Elementary
Barcelona Elementary
Kit Carson Elementary
Los Padillas Elementary
Mountain View Elementary
Navajo Elementary
Pajarito Elementary
Valle Vista Elementary
West Mesa High School
West Mesa High
Jimmy Carter Middle
John Adams Middle
Tres Volcanes (K-8)
Alamosa Elementary
Chaparral Elementary
Lavaland Elementary
M.A. Binford Elementary
Painted Sky Elementary
S.R. Marmon Elementary
Schools of Choice
Vision Quest
Table: Schools in the Northwest Quadrant
Cibola High School
Cibola High
James Monroe Middle
Taylor Middle
Corrales Elementary
Petroglyph Elementary
Seven Bar Elementary
Sierra Vista Elementary
Sunset View Elementary
Valley High School
Valley High
Garfield Middle
Taft Middle
Alameda Elementary
Alvarado Elementary
Cochiti Elementary
Duranes Elementary
Griegos Elementary
La Luz Elementary
Los Ranchos Elementary
MacArthur Elementary
Volcano Vista High School
Volcano Vista High
L.B. Johnson Middle
Tony Hillerman Middle
Chamiza Elementary
Marie Hughes Elementary
Tierra Antigua Elementary
Ventana Ranch Elementary
Table: Schools in the Northeast Quadrant
Del Norte High School
Del Norte High
Cleveland Middle
McKinley Middle
Arroyo del Oso Elementary
Bel-Air Elementary
E.G. Ross Elementary
Gov. Bent Elementary
Hodgin Elementary
Eldorado High School
Eldorado High
Eisenhower Middle
Hoover Middle
Georgia O'Keeffe Elementary
John Baker Elementary
Matheson Park Elementary
Mitchell Elementary
Onate Elementary
S.Y. Jackson Elementary
La Cueva High School
La Cueva High
Desert Ridge Middle
Dennis Chavez Elementary
Double Eagle Elementary
Hubert Humphrey Elementary
North Star Elementary
Sandia High School
Sandia High
Grant Middle
Madison Middle
Bellehaven Elementary
Comanche Elementary
Osuna Elementary
Sombra del Monte Elementary
"""

# Process each line
lines = [line.strip() for line in raw_text.strip().split("\n") if line.strip()]

cluster = None
quadrant = None
rows = []

for line in lines:
    # Detect quadrant headers
    if line.startswith("Table: Schools in the"):
        quadrant = line.replace("Table: Schools in the ", "").replace(" Quadrant", "")
    elif "High School" in line or "Schools of Choice" in line:
        cluster = line
    else:
        rows.append({
            "Quadrant": quadrant,
            "Cluster": cluster,
            "School": line
        })

# Create DataFrame
df = pd.DataFrame(rows)

In [3]:
df.head()

,Quadrant,Cluster,School
0,Southeast,Albuquerque High School,Albuquerque High
1,Southeast,Albuquerque High School,Jefferson Middle
2,Southeast,Albuquerque High School,Washington Middle
3,Southeast,Albuquerque High School,Dolores Gonzales Elementary
4,Southeast,Albuquerque High School,East San Jose Elementary


In [4]:
highland_schools_cluster = df[df["Cluster"] == "Highland High School"]
grouped = df.groupby("Cluster")["School"].apply(list).reset_index()
counts = df.groupby("Cluster")["School"].count().reset_index(name="Number of Schools")
grouped_on_quad = df.groupby(["Quadrant", "Cluster"])["School"].apply(list).reset_index()
northeast_quad_group = df[df["Quadrant"] == "Northeast"]
northwest_quad_group = df[df["Quadrant"] == "Northwest"]
southeast_quad_group = df[df["Quadrant"] == "Southeast"]
southwest_quad_group = df[df["Quadrant"] == "Southwest"]
quadrant_counts = df.groupby('Quadrant')['School'].count().reset_index(name='Number of Schools')
cluster_count = df.groupby("Cluster")["School"].count().reset_index(name="Number of Schools")

In [5]:
grouped.head()

,Cluster,School
0,Albuquerque High School,"[Albuquerque High, Jefferson Middle, Washingto..."
1,Atrisco Heritage High School,"[Atrisco Heritage Academy High, Truman Middle,..."
2,Cibola High School,"[Cibola High, James Monroe Middle, Taylor Midd..."
3,Del Norte High School,"[Del Norte High, Cleveland Middle, McKinley Mi..."
4,Eldorado High School,"[Eldorado High, Eisenhower Middle, Hoover Midd..."


In [6]:
grouped.dtypes

,0
Cluster,object
School,object


In [7]:
grouped_on_quad.head()

,Quadrant,Cluster,School
0,Northeast,Del Norte High School,"[Del Norte High, Cleveland Middle, McKinley Mi..."
1,Northeast,Eldorado High School,"[Eldorado High, Eisenhower Middle, Hoover Midd..."
2,Northeast,La Cueva High School,"[La Cueva High, Desert Ridge Middle, Dennis Ch..."
3,Northeast,Sandia High School,"[Sandia High, Grant Middle, Madison Middle, Be..."
4,Northwest,Cibola High School,"[Cibola High, James Monroe Middle, Taylor Midd..."


In [8]:
gis_url = 'https://coagisweb.cabq.gov/datadownload/apsschools.zip'
!curl -L -O {gis_url} > /dev/null 2>&1
!ls -la > /dev/null 2>&1
!unzip -l apsschools.zip > /dev/null 2>&1
!unzip apsschools.zip > /dev/null 2>&1

In [9]:
locations = 'schoolLocation.shp'
gpd.read_file(locations)

,OBJECTID,NAME,TYPE,GRADELEVEL,AUTHORIZER,SCHOOLOFCH,LASTUPDATE,created_us,created_da,last_edite,last_edi_1,geometry
0,1,The Montessori Elementary,Charter,Elementary/Middle,New Mexico Public Education Commission,None,2016-11-09,None,0000/00/00,None,0000/00/00,POINT (1517775.071 1506701.735)
1,2,Los Puentes,Charter,Middle/High,Albuquerque Public School District,None,2016-11-09,None,0000/00/00,None,0000/00/00,POINT (1522762.22 1500767.32)
2,3,Cesar Chavez Community School,Charter,High,New Mexico Public Education Commission,None,2016-11-09,None,0000/00/00,None,0000/00/00,POINT (1540567.697 1477134.032)
3,4,Gilbert L. Sena,Charter,High,New Mexico Public Education Commission,None,2016-11-09,None,0000/00/00,None,0000/00/00,POINT (1557692.471 1485845.595)
4,5,Southwest Learning Center,Charter,Elementary/Middle/High,New Mexico Public Education Commission,None,2016-11-09,None,0000/00/00,None,0000/00/00,POINT (1558065.135 1497936.448)
...,...,...,...,...,...,...,...,...,...,...,...,...
193,202,Helen Fox Education Complex,School of Choice,High,Albuquerque Public School District,None,2022-07-13,AGSMAG,2022/07/13,AGSMAG,2022/07/13,POINT (1526522.899 1478443.092)
194,203,Vision Quest at John Adams MS,School of Choice,Middle,Albuquerque Public School District,None,2022-07-13,AGSMAG,2022/07/13,AGSMAG,2022/07/13,POINT (1505514.755 1491464.732)
195,204,Nex+Gen,School of Choice,High,Albuquerque Public School District,None,2022-07-13,AGSMAG,2022/07/13,AGSMAG,2022/07/13,POINT (1541196.003 1503959.701)
196,205,Coyote Willow Family School,School of Choice,Elementary/Middle,Albuquerque Public School District,None,2022-07-13,AGSMAG,2022/07/13,AGSMAG,2022/07/13,POINT (1495479.017 1530114.005)


In [10]:
locations = 'schoolLocation.shp'
gdf = gpd.read_file(locations)
high = 'high.shp'
gdf = gpd.read_file(high)
gdf['GRADELEVEL']= 'high'
high_gdf = gdf.copy()
middle = 'middle.shp'
gdf = gpd.read_file(middle)
gdf['GRADELEVEL']= 'middle'
middle_gdf = gdf.copy()
elem = 'elem.shp'
gdf = gpd.read_file(elem)
gdf['GRADELEVEL']= 'elem'
elem_gdf = gdf.copy()

In [11]:
query = 'https://coagisweb.cabq.gov/arcgis/rest/services/public/APD_Incidents/MapServer/0/query?'
attr = '''
  where=1%3D1
  text=
  objectIds=
  time=
  geometry=
  geometryType=esriGeometryEnvelope
  inSR=
  spatialRel=esriSpatialRelIntersects
  relationParam=
  outFields=*
  returnGeometry=true
  maxAllowableOffset=
  geometryPrecision=
  outSR=4326
  returnIdsOnly=false
  returnCountOnly=false
  orderByFields=
  groupByFieldsForStatistics=
  outStatistics=
  returnZ=false
  returnM=false
  gdbVersion=
  returnDistinctValues=false
  f=pjson
'''.split()
incidents_url = query + "&".join(attr)

In [12]:
# Fetch the data and load it into a Python dictionary
!curl -L "{incidents_url}" > apd.incidents.json
with open("apd.incidents.json") as file:
    incidents_import = json.loads(file.read())

# Convert the features into a DataFrame
features = incidents_import['features']
incidents = pd.DataFrame(features)

# Extract relevant data into new columns
incidents['longitude'] = incidents['geometry'].apply(lambda entry: entry['x'])
incidents['latitude'] = incidents['geometry'].apply(lambda entry: entry['y'])
incidents['type'] = incidents['attributes'].apply(lambda entry: entry['IncidentType'])
incidents['address'] = incidents['attributes'].apply(lambda entry: entry['BlockAddress'])
incidents['datetime'] = incidents['attributes'].apply(lambda entry: entry['ReportDateTime'])



  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7810k  100 7810k    0     0   375k      0  0:00:20  0:00:20 --:--:-- 1824k


In [13]:
incidents_500 = incidents.sort_values( by = "datetime" ).iloc[-1500:]

In [14]:
all_schools = pd.concat([high_gdf, middle_gdf, elem_gdf], ignore_index=True)

In [15]:
all_schools

,OBJECTID,SCHOOLSERV,SCHOOLADDR,SCHOOLPHON,created_us,created_da,last_edite,last_edi_1,STAr,STLe,geometry,GRADELEVEL
0,1700,SANDIA,7801 CANDELARIA RD NE,298-0988,AGSMAG,2022-07-13,AGSMAG,2022-07-13,2.353271e+08,68741.397442,"POLYGON ((1547185.044 1512941.057, 1547371.752...",high
1,1701,DEL NORTE,5323 MONTGOMERY BLVD NE,881-9559,AGSMAG,2022-07-13,AGSMAG,2022-07-13,3.512029e+08,105305.246749,"POLYGON ((1537128.805 1518952.828, 1537234.007...",high
2,1702,VALLEY,1505 CANDELARIA RD NW,345-9021,AGSMAG,2022-07-13,AGSMAG,2022-07-13,6.871206e+08,139689.939494,"POLYGON ((1533786.113 1518904.367, 1533634.705...",high
3,1703,ALBUQUERQUE,800 ODELIA RD NE,843-6400,AGSMAG,2022-07-13,AGSMAG,2022-07-13,1.354403e+09,211961.441896,"POLYGON ((1527029.821 1494037.171, 1528606.462...",high
4,1704,LA CUEVA,7801 WILSHIRE AV NE,821-2410,AGSMAG,2022-07-13,AGSMAG,2022-07-13,1.021756e+09,153180.460005,"POLYGON ((1545120.334 1534703.535, 1545304.099...",high
...,...,...,...,...,...,...,...,...,...,...,...,...
126,1451,EDWARD GONZALES,594 90TH ST SW,831-6214,AGSMAG,2022-07-13,AGSMAG,2022-07-13,7.066998e+07,41582.113923,"POLYGON ((1502693.737 1479671.603, 1503128.277...",elem
127,1452,HELEN CORDERO,8800 EUCARIZ AV SW,833-5830,AGSMAG,2022-07-13,AGSMAG,2022-07-13,7.066998e+07,41582.113923,"POLYGON ((1502693.737 1479671.603, 1503128.277...",elem
128,1453,NAVAJO,2936 HUGHES RD SW,873-8512,AGSMAG,2022-07-13,AGSMAG,2022-07-13,1.161912e+08,67893.953583,"POLYGON ((1505045.153 1472090.176, 1505889.002...",elem
129,1454,BARCELONA,2311 BARCELONA RD SW,877-0400,AGSMAG,2022-07-13,AGSMAG,2022-07-13,4.441823e+07,32364.417385,"POLYGON ((1506257.281 1471063.589, 1506287.057...",elem


In [16]:
grouped.head()

,Cluster,School
0,Albuquerque High School,"[Albuquerque High, Jefferson Middle, Washingto..."
1,Atrisco Heritage High School,"[Atrisco Heritage Academy High, Truman Middle,..."
2,Cibola High School,"[Cibola High, James Monroe Middle, Taylor Midd..."
3,Del Norte High School,"[Del Norte High, Cleveland Middle, McKinley Mi..."
4,Eldorado High School,"[Eldorado High, Eisenhower Middle, Hoover Midd..."


In [17]:
len(grouped.Cluster.unique())

14

In [18]:
all_schools.head()

,OBJECTID,SCHOOLSERV,SCHOOLADDR,SCHOOLPHON,created_us,created_da,last_edite,last_edi_1,STAr,STLe,geometry,GRADELEVEL
0,1700,SANDIA,7801 CANDELARIA RD NE,298-0988,AGSMAG,2022-07-13,AGSMAG,2022-07-13,2.353271e+08,68741.397442,"POLYGON ((1547185.044 1512941.057, 1547371.752...",high
1,1701,DEL NORTE,5323 MONTGOMERY BLVD NE,881-9559,AGSMAG,2022-07-13,AGSMAG,2022-07-13,3.512029e+08,105305.246749,"POLYGON ((1537128.805 1518952.828, 1537234.007...",high
2,1702,VALLEY,1505 CANDELARIA RD NW,345-9021,AGSMAG,2022-07-13,AGSMAG,2022-07-13,6.871206e+08,139689.939494,"POLYGON ((1533786.113 1518904.367, 1533634.705...",high
3,1703,ALBUQUERQUE,800 ODELIA RD NE,843-6400,AGSMAG,2022-07-13,AGSMAG,2022-07-13,1.354403e+09,211961.441896,"POLYGON ((1527029.821 1494037.171, 1528606.462...",high
4,1704,LA CUEVA,7801 WILSHIRE AV NE,821-2410,AGSMAG,2022-07-13,AGSMAG,2022-07-13,1.021756e+09,153180.460005,"POLYGON ((1545120.334 1534703.535, 1545304.099...",high


In [19]:
locations

'schoolLocation.shp'

In [20]:
# Define a list of colors to choose from
color_palette = [
    'green', 'red', 'black', 'yellow', 'blue', 'purple', 'orange', 'brown',
    'grey', 'pink', 'cyan', 'magenta', 'lime', 'indigo', 'teal', 'violet',
    'turquoise', 'silver', 'gold', 'navy'
]


# Create a dictionary to map each cluster to a color
cluster_colors = {cluster: color_palette[i % len(color_palette)]
                  for i, cluster in enumerate(grouped['Cluster'].unique())}

# Create a dictionary for school name and its corresponding cluster color
school_colors = {}

# Iterate over the grouped DataFrame to map each school to its corresponding cluster color
for _, row in grouped.iterrows():
    cluster = row['Cluster']
    color = cluster_colors.get(cluster, "gray")  # Default to gray if no color found
    for school in row['School']:
        school_colors[school] = color

# Now 'school_colors' dictionary contains the school names as keys and their cluster colors as values
print(school_colors)


{'Albuquerque High': 'green', 'Jefferson Middle': 'green', 'Washington Middle': 'green', 'Dolores Gonzales Elementary': 'green', 'East San Jose Elementary': 'green', 'Eugene Field Elementary': 'green', 'Lew Wallace Elementary': 'green', 'Longfellow Elementary': 'green', 'Lowell Elementary': 'green', 'Monte Vista Elementary': 'green', 'Montezuma Elementary': 'green', 'Reginald Chavez Elementary': 'green', 'Zia Elementary': 'green', 'Atrisco Heritage Academy High': 'red', 'Truman Middle': 'red', 'Rudolfo Anaya Elementary': 'red', 'Helen Cordero Elementary': 'red', 'Edward Gonzales Elementary': 'red', 'Carlos Rey Elementary': 'red', 'George I. Sánchez (K-8)': 'red', 'Cibola High': 'black', 'James Monroe Middle': 'black', 'Taylor Middle': 'black', 'Corrales Elementary': 'black', 'Petroglyph Elementary': 'black', 'Seven Bar Elementary': 'black', 'Sierra Vista Elementary': 'black', 'Sunset View Elementary': 'black', 'Del Norte High': 'yellow', 'Cleveland Middle': 'yellow', 'McKinley Middle':

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def best_match_key(dictionary, query):
    # Create a TfidfVectorizer
    vectorizer = TfidfVectorizer()

    # Combine the query with all the keys from the dictionary to compute the similarity
    keys = list(dictionary.keys())
    all_text = keys + [query]  # Append query to the list of keys

    # Convert the text to TF-IDF vectors
    tfidf_matrix = vectorizer.fit_transform(all_text)

    # Calculate cosine similarity between the query and all keys
    cosine_similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])

    # Return the key with the highest similarity
    return keys[cosine_similarities.argmax()]

# Example usage:
best_match_key(school_colors, "Jefferson")



'Jefferson Middle'

In [22]:
from folium.plugins import MarkerCluster
import folium
import geopandas as gpd

# Assuming 'locations' is the file path to your shapefile
# Read the shapefile and ensure it's in EPSG:4326
locationsgpd = gpd.read_file(locations)  # If locations is a file path, read the shapefile
locationsgpd = locationsgpd.to_crs(epsg=4326)  # Ensure CRS is EPSG:4326

# Initialize the map (centered based on your area of interest, or set a default)
m = folium.Map(location=[35.0844, -106.6504], zoom_start=11)
marker_cluster = MarkerCluster().add_to(m)

# Add school markers using the 'geometry' column in the locations GeoDataFrame
for _, row in locationsgpd.iterrows():

    # Get the latitude and longitude from the geometry
    point = row['geometry']
    if point.geom_type == 'Point':  # Check if it's a point geometry
        folium.Marker(
            location=[point.y, point.x],  # Latitude, Longitude
            popup=f"{row['NAME']} ({row['GRADELEVEL']})",  # Adjusted to use 'NAME'
            #icon=folium.Icon(color="blue" if "High" in row["GRADELEVEL"] else "green" if "Middle" in row["GRADELEVEL"] else "orange")
            icon=folium.Icon(color=school_colors[best_match_key(school_colors,row["NAME"])])

        ).add_to(marker_cluster)

# Create GeoDataFrame for incidents (ensure lat/lon are in correct order)
gdf_incidents = gpd.GeoDataFrame(
    incidents_500,
    geometry=gpd.points_from_xy(incidents_500["longitude"], incidents_500["latitude"]),
    crs="EPSG:4326"  # Ensure incidents are in the correct CRS as well
)

# Add incident markers
for _, row in gdf_incidents.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=5,
        color="red",
        fill=True,
        fill_opacity=0.6,
        popup=row["type"]
    ).add_to(marker_cluster)

# Display the map
m


<ipython-input-22-ba0805837fc9>:24: UserWarning: color argument of Icon should be one of: {'black', 'orange', 'beige', 'darkred', 'lightred', 'darkblue', 'blue', 'lightgreen', 'lightblue', 'white', 'darkgreen', 'darkpurple', 'pink', 'cadetblue', 'purple', 'green', 'red', 'gray', 'lightgray'}.
  icon=folium.Icon(color=school_colors[best_match_key(school_colors,row["NAME"])])
